<a href="https://colab.research.google.com/github/dragdowl1/s1_devices/blob/main/Yulia_Draginda/Internship_pass_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install easyocr

In [2]:
%%capture
!git clone https://github.com/ultralytics/yolov5 
%cd yolov5
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [3]:
import os  
import cv2
import math
import torch
import keras
import pickle
import warnings
import torchvision
import numpy as np
import pandas as pd
import tensorflow as tf
from google.colab import drive 
from keras.models import load_model
from torch.autograd import Variable
from google.colab.patches import cv2_imshow
from IPython.display import Image, clear_output 

In [4]:
%cd /content
drive.mount('/content/gdrive/')

/content
Mounted at /content/gdrive/


In [63]:
class OCR_Model(torch.nn.Module):
    def __init__(self, n_classes=53, out_len=32):
        super().__init__()

        self.m = torchvision.models.resnet34(weights='ResNet34_Weights.IMAGENET1K_V1')
        self.blocks = [torch.nn.Conv2d(3, 64, 7, 1, 3), self.m.bn1, self.m.relu, self.m.maxpool, 
                      self.m.layer1, self.m.layer2, self.m.layer3]
        self.feature_extractor = torch.nn.Sequential(*self.blocks)
        self.avg_pool = torch.nn.AdaptiveAvgPool2d((512, out_len))
        self.bilstm1 = torch.nn.LSTM(512, 256, 2, dropout=0.15, batch_first=True, bidirectional=True)
        self.bilstm2 = torch.nn.LSTM(512, 256, 2, dropout=0.15, batch_first=True, bidirectional=True)
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(512, 256),
            torch.nn.GELU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(256, n_classes))
        

    def forward(self, x, return_x=False):

        feature = self.feature_extractor(x)
        b, c, h, w = feature.size()
        feature = feature.view(b, c * h, w)
        feature = self.avg_pool(feature)
        feature = feature.transpose(1, 2)
        out, (h_t1, c_t1) = self.bilstm1(feature)
        out, _ = self.bilstm2(out, (h_t1, c_t1))
        #print(x.shape)
        out = self.classifier(out)
        
        return out

class Passport():
    
    def __init__(self):
        self.ModelFolder=''#"OCRModels/"
        self.Model1=load_model(self.ModelFolder+'OCRModels/Model_w740_h64_zeroz1312-740_dropout_numbers_v2.h5', compile=False)
        self.Model2=load_model(self.ModelFolder+'OCRModels/Model_w740_h64_zeroz1312-740_dropout_v2.h5', compile=False)
        #self.Model3=load_model(self.ModelFolder+'OCRModels/conv2d_v8_model.h5')
        
        self.model_round = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/rotation.pt', source='local')
        self.model_detect = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/detect.pt', source='local')
        self.Model1_2_img_width = 1312  #1148  # 750
        self.Model1_2_img_height = 64  #164   # 64
        self.Model1_3_img_width = 100
        self.Model1_3_img_height = 50
        with open(self.ModelFolder+"OCRModels/characters_letters.pickle", 'rb') as f:
            Model1_2_characters= pickle.load(f)
        char_to_num = keras.layers.StringLookup(vocabulary=list(Model1_2_characters), mask_token=None)
        self.num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)
        self.MaxLenLabel_1_2=80

        #юлина модель
        self.Model4_short = OCR_Model(out_len=32)
        self.Model4_short.load_state_dict(torch.load(self.ModelFolder+'OCRModels/best_short.pt',map_location=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))) 
        self.Model4_long = OCR_Model(out_len=80)
        self.Model4_long.load_state_dict(torch.load(self.ModelFolder+'OCRModels/best_long.pt',map_location=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')))
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
   #юлина модель
    def text2ind(self, text):
        inds = []
        char_list = ['№', '’', 'Я', 'Ю', 'Э', 'Ь', 'Ы', 'Ъ', 'Щ', 'Ш', 'Ч', 'Ц', 'Х', 'Ф', 'У', 'Т', 'С', 'Р', 'П', 'О', 'Н', 'М', 'Л', 'К', 'Й', 'И', 'З', 'Ж', 'Е', 'Д', 'Г', 'В', 'Б', 'А', 'Ё', '|', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0', '/', '.', '-', ')', '(', '"']
        for c in text:
            if c in char_list:
                ch = char_list.index(c)
            else:
                ch = 0
            inds.append(int(ch))
        return inds

    def inds2text(self, labels):
        string = []
        char_list = ['№', '’', 'Я', 'Ю', 'Э', 'Ь', 'Ы', 'Ъ', 'Щ', 'Ш', 'Ч', 'Ц', 'Х', 'Ф', 'У', 'Т', 'С', 'Р', 'П', 'О', 'Н', 'М', 'Л', 'К', 'Й', 'И', 'З', 'Ж', 'Е', 'Д', 'Г', 'В', 'Б', 'А', 'Ё', '|', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0', '/', '.', '-', ')', '(', '"']
        for lbl in labels:
          string.append(char_list[int(lbl)])
        return ''.join(string)

    # картинки
    def black2white(self, image):
        
        lo=np.array([0,0,0])
        hi=np.array([0,0,0])
        mask = cv2.inRange(image, lo, hi)
        image[mask>0]=(255,255,255)

        return image

    def process_image(self, image, xdim):
      im = self.black2white(image) 
      im = im.astype(np.float32) / 255
      im = cv2.resize(im, (xdim, 64), interpolation=cv2.INTER_LINEAR) # reshape
      im = np.moveaxis(im, -1, 0)
      X = torch.FloatTensor(im).type(torch.FloatTensor).unsqueeze(0) # добавить размерность батча
      return X

    # функция предикта
    def predict_geo(self, image, ocr_model, xdim, delimiter = '|', collapse_repeated=True):
      ocr_model.to(self.device)
      X = self.process_image(image, xdim).to(self.device)  
      char_list = ['№', '’', 'Я', 'Ю', 'Э', 'Ь', 'Ы', 'Ъ', 'Щ', 'Ш', 'Ч', 'Ц', 'Х', 'Ф', 'У', 'Т', 'С', 'Р', 'П', 'О', 'Н', 'М', 'Л', 'К', 'Й', 'И', 'З', 'Ж', 'Е', 'Д', 'Г', 'В', 'Б', 'А', 'Ё', '|', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0', '/', '.', '-', ')', '(', '"']
        
      with torch.no_grad():
        ocr_model.eval()
        output = torch.nn.functional.log_softmax(ocr_model(X), dim=-1).to(torch.float32)
        arg_maxes = torch.argmax(output, dim=2)
        decodes = []
        for i, args in enumerate(arg_maxes):
            dec = []
            str_out = []

            for j, index in enumerate(args):
                if index != 52:
                    if collapse_repeated and j != 0 and index == args[j -1]:
                        continue
                    dec.append(index.item())

            for d in dec:
              str_out.append(char_list[int(d)])     
            decodes.append(''.join(str_out))

        preds =  [tp.replace('|', " ").strip() for tp in decodes] # fatal!!:))  
        b_greedy_transcript = preds[0]
      

      return b_greedy_transcript

    def get_geo_transcript(self, image,  xdim1 = 256, xdim2 = 640):
        ocr_model1=self.Model4_short
        ocr_model2=self.Model4_long
        transcript = self.predict_geo(image, ocr_model1, xdim1)
        if len(transcript)>=15:
            transcript = self.predict_geo(image, ocr_model2, xdim2)
        return transcript

    def encode_mw(self,img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (self.Model1_3_img_width, self.Model1_3_img_height), cv2.INTER_NEAREST)
        img1 = np.array(img)
        img1 = img1.reshape(img1.shape[0],img1.shape[1],1)
        prd=self.Model3.predict(img1)
  
        self.Model1_2_
    def encode_single_sample(self,img,ConvertionType=0):#type1=0 - Numbers  type=1 - Text
        # 1. Read image
        #img = tf.io.read_file(img_path)
        # 2. Decode and convert to grayscale
        #img = tf.io.decode_jpeg(img, channels=1)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 3. Convert to float32 in [0, 1] range
        img = np.expand_dims(img,axis=-1)
	
        img1=img.copy()
        img1 = tf.image.convert_image_dtype(img1, tf.float32)
        #print(type(img1),img1.shape)
        # 4. Resize to the desired size
        # h, w, ch = img.shape.as_list()
        # img_width = int(w * img_height / h)
        img1 = tf.image.resize(img1, [self.Model1_2_img_height, 740])
        #print(type(img1),img1.shape)
        # 4.1
        # h, w, ch = img.shape.as_list()
        # ones = tf.ones((h, (1312 - w), ch), dtype=tf.float32)
        ones = tf.ones((self.Model1_2_img_height, (self.Model1_2_img_width - 740), 1), dtype=tf.float32)
        img1 = tf.concat([img1, ones], 1)
        # 5. Transpose the image because we want the time
        # dimension to correspond to the width of the image.
        img1 = tf.transpose(img1, perm=[1, 0, 2])
        # 6. Map the characters in label to numbers
        #label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
	
        if(ConvertionType==0):
            Prediction=self.Model1.predict(np.expand_dims(img1,0),verbose=0)#np.expand_dims(ImArr,0)
        elif(ConvertionType==1):
            Prediction=self.Model2.predict(np.expand_dims(img1,0),verbose=0)
			
        input_len = np.ones(Prediction.shape[0]) * Prediction.shape[1]
        # Use greedy search. For complex tasks, you can use beam search
        results = keras.backend.ctc_decode(Prediction, input_length=input_len, greedy=True)[0][0][:,:self.MaxLenLabel_1_2]
        output_text=tf.strings.reduce_join(self.num_to_char(results[0])).numpy().decode("utf-8")
        output_text=output_text.replace("[UNK]","")
        #print(output_text)
        BreaksEnd=False
        BreaksEndPos=0
        for i in range(-1,-len(output_text),-1):
            if(output_text[i]!=" " and BreaksEndPos==0):
                BreaksEnd=True
                BreaksEndPos=i
                break
            #print(BreaksEndPos)
        if(BreaksEndPos<-1):
            output_text=output_text[:BreaksEndPos+1]
        #print("output_text",output_text)
        return output_text
		
		
    def zero(self,n):
        return n * (n > 0)

    def rotate_image(self, mat, angle):
        height, width = mat.shape[:2]
        image_center = (width / 2, height / 2)
        rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
        abs_cos = abs(rotation_mat[0, 0])
        abs_sin = abs(rotation_mat[0, 1])
        bound_w = int(height * abs_sin + width * abs_cos)
        bound_h = int(height * abs_cos + width * abs_sin)
        rotation_mat[0, 2] += bound_w / 2 - image_center[0]
        rotation_mat[1, 2] += bound_h / 2 - image_center[1]
        return cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))

    def get_angle_rotation(self, centre, point, target_angle):
        new_point =(point[0] - centre[0], point[1] - centre[1])
        a,b = new_point[0], new_point[1]
        res = math.atan2(b,a)
        if (res < 0) :
              res += 2 * math.pi
        return (math.degrees(res)+target_angle) % 360

    def result(self,img):
        return self.model_round(img)

    def get_image_after_rotation(self, img):
        results = self.result(img)
        pd = results.pandas().xyxy[0]
        pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)
        pd = pd.assign(centre_y = pd.ymin + (pd.ymax-pd.ymin)/2)
        tmp = pd.loc[pd['name']=='niz']
        N, V = None, None
        for index, row in tmp.iterrows():
            N = (row['centre_x'], row['centre_y'])
            break
        tmp = pd.loc[pd['name']=='verh']
        for index, row in tmp.iterrows():
            V = (row['centre_x'], row['centre_y'])
            break
        if N == None or V == None:
            return img

        angle = self.get_angle_rotation(N, V, 90)
        img = self.rotate_image(img, angle)
        return img

    def crop_img(self, img):
        #print(img.size)
        results = self.result(img)
        #print(results.pandas())
        pd = results.pandas().xyxy[0]
        x1 =int(pd.xmin.min())
        x2 = int(pd.xmax.max())
        y1 = int(pd.ymin.min())
        y2 = int(pd.ymax.max())
        img = img[y1:y2,x1:x2]
        return img

    def get_crop(self,file):
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = self.get_image_after_rotation(image)
        image = self.get_image_after_rotation(image)
        image = self.crop_img(image)
        return image

    def yolo_5(self, img):
        results = self.model_detect(img)
        df = results.pandas().xyxy[0]
        df = df.drop(np.where(df['confidence'] < 0.7)[0])
        # print(df)
        ob = pd.DataFrame()
        ob['class'] = df['name']
        ob['x'] = df['xmin']
        ob['y'] = df['ymin']
        ob['w'] = df['xmax']-df['xmin']
        ob['h'] = df['ymax']-df['ymin']
        oblasty = ob.values.tolist()
        return img, oblasty

    def oblasty_yolo_5(self, image, box):
        oblasty = {}
        iss = 0
        plac = 0
        spissok = sorted(box, reverse=False, key=lambda x: x[2])
        for l in spissok:
            cat = l[0]
            y = int(l[2])
            x = int(l[1])
            h = int(l[4])
            w = int(l[3])
            ob = ''
            if ('signature' in cat) or ('photograph' in cat):
                pass
            else:
                if 'issued_by_whom' in cat:
                    ob = cat + '_' + str(iss)
                    iss += 1
                elif 'place_of_birth' in cat:
                    ob = cat + '_' + str(plac)
                    plac += 1
                elif 'series' not in cat:
                    ob = cat
                if ob:
                    oblasty[ob] = image[self.zero(y - math.ceil(h * 0.07)):y + math.ceil(h * 1.1),
                                  self.zero(x - math.ceil(w * 0.1)):x + math.ceil(w * 1.1)]
                if 'series' in cat:
                    ob = cat
                    cropped = image[self.zero(y - math.ceil(h * 0.1)):y + math.ceil(h * 1.1),
                              self.zero(x - math.ceil(w * 0.03)):x + math.ceil(w * 1.03)]
                    oblasty[ob] = cv2.rotate(cropped, cv2.ROTATE_90_COUNTERCLOCKWISE) #self.rotate_image(cropped, 90)
        return oblasty

    def recognition_slovar(self, oblasty):
        data = {}
        data['pasport'] = []
        d = {}
        issued_by_whom = ''
        series_and_number = ''
        place_of_birth = ''
        ver = 0
        for i, v in oblasty.items():
            image = cv2.cvtColor(v, cv2.COLOR_BGR2RGB)
            # if image.shape[0] <=23:
            #     image = self.resiz(image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Для каждого класса устанавливаем свои ограничения на распознания классов
            #print(i,np.array(image).shape)
            if 'date' in i:
                result = self.encode_single_sample(image, ConvertionType=0)
            elif 'code' in i:
                result = self.encode_single_sample(image, ConvertionType=0)
            elif 'series' in i:
                result = self.encode_single_sample(image, ConvertionType=0)
            elif 'surname' in i or 'name' in i or 'patronymic' in i:
                result = self.get_geo_transcript(image)
                # result = self.encode_single_sample(image, ConvertionType=1)
            elif 'issued_by_whom' in i:
                result = self.get_geo_transcript(image)
                # print(result)
                # cv2_imshow(image)
            elif 'place_of_birth' in i:
                result = self.get_geo_transcript(image)
                # print(result)
            elif 'gender' in i:
                result = self.encode_single_sample(image, ConvertionType=1)

                                                    
            else:
                result = self.encode_single_sample(image,ConvertionType=1)
            pole = result#''
            """
            for k in range(len(result)):
                if result[k][2] * 100 >= 25:
                    if str(result[k][1]).isnumeric():
                        if result[k][2] * 100 >= 70:
                            pole = pole + ' ' + str(result[k][1])
                    else:
                            pole = pole + ' ' + str(result[k][1])
            """
            if pole:
                pole = pole.strip()
                if 'issued_by_whom' in i:
                    issued_by_whom = issued_by_whom + pole + ' '
                if 'place_of_birth' in i:
                    place_of_birth = place_of_birth + pole + ' '
                if 'series_and_number' in i:
                    # print(pole)
                    """
                    if len(pole) >= 10:
                        if ver < result[k][2]:
                            series_and_number = pole
                    """
                    series_and_number = pole
                    
                if 'issued_by_whom' in i or 'place_of_birth' in i or 'series_and_number' in i:
                    pass
                elif 'date' in i:
                    pole = pole.replace('.', '').replace(' ', '').replace('-', '')
                    pole = pole[:2] + '.' + pole[2:4] + '.' + pole[4:8]
                    d[i.split('.', 1)[0]] = pole.upper().strip()
                elif 'cod' in i:
                    pole = pole.replace(' . ', '').replace(' ', '').replace('-', '')
                    pole = pole[:3] + '-' + pole[3:6]
                    d[i.split('.', 1)[0]] = pole.upper().strip()
                elif 'gender' in i:
                    pole1 = ''
                    if 'Е' in pole.upper() or 'Н' in pole.upper() or 'ЕН' in pole.upper():
                        pole1 = 'ЖЕН.'
                    if 'У' in pole.upper() or 'М' in pole.upper() or 'УЖ' in pole.upper():
                        pole1 = 'МУЖ.'
                    pole = pole1
                    d[i.split('.', 1)[0]] = pole.upper().strip()
                else:
                    d[i.split('.', 1)[0]] = pole.replace('  ', ' ').upper().strip()

        place_of_birth = place_of_birth.upper()
        issued_by_whom = issued_by_whom.upper()
        
        if place_of_birth[:2] == 'C ':
            place_of_birth = place_of_birth.replace('С ', ' С. ')
        if issued_by_whom[:2] == 'C ':
            issued_by_whom = issued_by_whom.replace('С ', ' С. ')

        place_of_birth = place_of_birth.replace('ГОР ', 'ГОР. ')\
                                        .replace(' С ', ' С. ')\
                                        .replace(' Г ', ' Г. ')\
                                        .replace('ОБЛ ', 'ОБЛ. ')\
                                        .replace('ПОС ', 'ПОС. ')\
                                        .replace('ДЕР ', 'ДЕР. ')\
                                        .replace(' . ', '. ')\
                                        .replace(' .', '.')\
                                        .replace('  ', ' ')\
                                        .replace('..', '.')\
                                        .replace('.', '. ')\
                                        .replace('  ', ' ')\
                                        .replace('--', '-')

        issued_by_whom = issued_by_whom.replace('ГОР ', 'ГОР. ')\
                                        .replace(' С ', ' С. ')\
                                        .replace(' Г ', ' Г. ')\
                                        .replace('ОБЛ ', 'ОБЛ. ')\
                                        .replace('ПОС ', 'ПОС. ')\
                                        .replace('ДЕР ', 'ДЕР. ')\
                                        .replace(' . ', '. ')\
                                        .replace(' .', '.')\
                                        .replace('  ', ' ')\
                                        .replace('..', '.')\
                                        .replace('.', '. ')\
                                        .replace('  ', ' ')\
                                        .replace('--', '-')
        if series_and_number:
            series_and_number = series_and_number.replace(' ', '')
            if len(series_and_number) == 10:
                series_and_number = series_and_number[:2] + ' ' + series_and_number[2:4] + ' ' + series_and_number[4:10]
            else:
                series_and_number = 'поле распознано не полностью' + series_and_number
        else:
            series_and_number = ''
        d['issued_by_whom'] = issued_by_whom.strip()
        d['place_of_birth'] = place_of_birth.strip()
        d['series_and_number'] = series_and_number.strip()
        data['pasport'].append(d)
        return data['pasport']

    def detect_passport(self,photo):
        pole = ['date_of_birth','date_of_issue','first_name','gender','issued_by_whom',
                'patronymic','place_of_birth','series_and_number','surname','unit_code']

        croped = self.get_crop(photo)
        if croped != '':
            img, detect = self.yolo_5(croped)
            obl = self.oblasty_yolo_5(img, detect)
            rec = self.recognition_slovar(obl)
            key = list(rec[0].keys())
            value = list(rec[0].values())
            if set(key) == set(pole):
                if '' in value:
                    flag = 1

                else:
                    flag = 0
                    if len(rec[0]['date_of_birth']) != 10 or len(rec[0]['date_of_issue']) != 10 or len(rec[0]['series_and_number']) != 12 or len(rec[0]['unit_code']) != 7:
                        flag = 1
            else:
                flag = 1

            if flag == 1:
                rec =rec[0]# {}
            else:
                rec = rec[0]
            return rec, flag
        else:
            return {}, 1

     

In [6]:
!ls /content/gdrive/MyDrive/ClassAssemble

OCRModels	     passport_class.py	requirements.txt  yolov5_master
passport_class_2.py  __pycache__	yolo5


In [18]:
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
# from gdrive.MyDrive.ClassAssemble.passport_class import Passport,INN,Snils

In [7]:
!cp -R /content/gdrive/MyDrive/ClassAssemble/yolo5 yolo5
!cp -R /content/gdrive/MyDrive/ClassAssemble/yolov5_master yolov5_master
!cp -R /content/gdrive/MyDrive/ClassAssemble/OCRModels OCRModels

In [64]:
psprt = Passport()

YOLOv5 🚀 2023-5-2 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 290 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 2023-5-2 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 322 layers, 86254162 parameters, 0 gradients, 204.0 GFLOPs
Adding AutoShape... 


In [9]:
!unzip /content/test_pass.zip
clear_output()

In [65]:
folder_path = '/content/test_pass'
results=[]
for file_name in os.listdir(folder_path):
    if file_name.endswith('.jpeg'):
        file_path = os.path.join(folder_path, file_name)
        data1=psprt.detect_passport(file_path)
        results.append((file_name, data1))

In [66]:
df_data = []
for tup in results:
    row_dict = {}
    row_dict['file_name'] = tup[0]
    for k, v in tup[1][0].items():
        row_dict[k] = v
    df_data.append(row_dict)
df1 = pd.DataFrame(df_data)
df1 = df1.sort_values(by='file_name').reset_index(drop=True)
df1['issued_by_whom'] = df1['issued_by_whom'].astype(str)
df1['issued_by_whom'] = df1['issued_by_whom'].str.strip()
df1['place_of_birth'] = df1['place_of_birth'].astype(str)
df1['place_of_birth'] = df1['place_of_birth'].str.strip()
df1['gender'] = df1['gender'].astype(str)
df1['gender'] = df1['gender'].str.strip()

df1

,file_name,unit_code,date_of_issue,surname,first_name,patronymic,gender,date_of_birth,issued_by_whom,place_of_birth,series_and_number
0,0c5cd9f2-96c9-43ed-9008-f6de3b1a8a8c.jpeg,380-003,02.07.2013,НЫРКО,АНАСТАСИЯ,МАТВЕЕВНА,ЖЕН.,07.05.1991,ОТДЕЛОМ УФМС РОССИИ ПО ИРКУТСКОЙ ОБЛАСТИ В ОКТ...,ГОР. ИРКУТСК,25 13 856830
1,177de6d27597c51ac801fe8bab8e0931.jpeg,230-029,73.01.2012,БАТИЩЕВ,ВЯЧЕСЛАВ,ВЯЧЕСЛАВОВИЧ,МУЖ.,19.12.1991,ОУФМС РОССИИ ПО КРАСНОДАРСКОМУ КРАЮ В БЕЛОГЛИН...,ГОР. МАЙКОП РЕСП. АДЫГЕЯ,03 11 968450
2,27f2fdc8b1ffc41d481b57ab8b568917.jpeg,630-027,06.07.2022,ДЖУМАЕВ,АХОМИДИН,САЛОМУДИНОВИЧ,МУЖ.,30.04.1990,ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛАСТИ,С. АЙНИ КУМСАНГИРСКИЙ Р-Н КУРГАН-ТЮБИНСКАЯ ОБЛ...,36 22 110830
3,3672a1e729a3e5371b9ec45c5a91c297.jpeg,632-010,28.09.2002,РУБАНОВ,ДМИТРИЙ,ГЕОРГИЕВИЧ,МУЖ.,20.11.1979,ОКТЯБРЬСКИМ РОВД ГОРОДА САМАРЫ,Г. КУЙБЫШЕВ,36 02 551539
4,3710bc70ed198d96abb3f7e8e373276c.jpeg,502-015,15.02.2002,ОПРЯ,ДМИТРИЙ,СТЕПАНОВИЧ,МУЖ.,07.08.1979,ЛОБНЕНСКИМ ОВД МЫТИЩИНСКОГО УВД МОСКОВСКОЙ ОБЛ...,Г. ЛОБНЯ МОСКОВСКОЙ ОБЛ.,поле распознано не полностью46024527213
5,44ab4177f18cd5dd8a898c852c5fd2c4.jpeg,550-038,24.07.2009,ГУДЫМ,МАКСИМ,ВАЛЕРЬЕВИЧ,МУЖ.,30.09.1984,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО ОМСКОЙ ОБЛАСТИ В ЧЕР...,С. МИХАЙЛОВКА ЧЕРЛАКСКОГО Р-НА ОМСКОЙ ОБЛ.,52 09 789231
6,5139349d9409170acd30abcf996f09cf.jpeg,640-011,15.06.2015,БАЛТАДЖЯН,АРКАДИЙ,СЕРГЕЕВИЧ,МУЖ.,04.07.1956,ОТДЕЛОМ УФМС РОССИИ ПО САРАТОВСКОЙ ОБЛАСТИ В Г...,ГОР. ТБИЛИСИ Г. ССР,63 15 160426
7,582b6e51f655e69e33dcbb6049cae588.jpeg,162-013,01.04.2002,БАШАН,АЛЕКСЕЙ,АЛЕКСАНДРОВИЧ,МУЖ.,26.07.1981,ЭЛЕКТРОТЕХНИЧЕСКИМ ОВД ГОР. НАБЕРЕЖНЫЕ ЧЕЛНЫ Р...,ГОР. КАЗАНЬ,92 02 983452
8,68418534-4c5f-43db-8277-37b9596d31ae.jpeg,360-004,07.12.2017,КУРАСОВ,АЛЕКСАНДР,ПЕТРОВИЧ,МУЖ.,06.04.1971,ОТДЕЛОМ УФМС РОССИИ ПО ВОРОНЕЖСКОЙ ОБЛАСТИ В К...,С. Б-ПЕСКИ ЭРТИЛЬСКОГО Р-НА ВОРОНЕЖСКОЙ ОБЛ.,20 17 158065
9,7d8c2607-7bd0-47c9-8011-e035a6f34dd4.jpeg,780-001,25.06.2007,ЕГОРОВ,ВАСИЛИЙ,АЛЕКСАНДРОВИЧ,МУЖ.,20.04.1987,ТП №1 ОТДЕЛА УФМС РОССИИ ПО АДМИРАЛТЕЙСКОМ Р-Н...,ГОР. ЛЕНИНГРАД,40 07 187829


In [67]:
df = pd.read_excel('/content/test_pass/test_pass_labels.xlsx', na_filter=False)
df.fillna('', inplace=True)
df['issued_by_whom'] = df['issued_by_whom_0'].apply(lambda x: str(x)) + ' ' + df['issued_by_whom_1'].apply(lambda x: str(x)) + ' ' + df['issued_by_whom_2'].apply(lambda x: str(x))
df['place_of_birth'] = df['place_of_birth_0'].apply(lambda x: str(x)) + ' ' + df['place_of_birth_1'].apply(lambda x: str(x)) + ' ' + df['place_of_birth_2'].apply(lambda x: str(x))
df.rename(columns={'series_and_number_1': 'series_and_number'}, inplace=True)
df2 = df.drop(columns=['issued_by_whom_0', 'issued_by_whom_1', 'issued_by_whom_2', 'place_of_birth_0', 'place_of_birth_1', 'place_of_birth_2', 'series_and_number_2'])
df2

,file_name,date_of_issue,unit_code,surname,first_name,patronymic,date_of_birth,gender,series_and_number,issued_by_whom,place_of_birth
0,0c5cd9f2-96c9-43ed-9008-f6de3b1a8a8c.jpeg,02.07.2013,380-003,НЫРКО,АНАСТАСИЯ,МАТВЕЕВНА,07.05.1991,ЖЕН.,25 13 856830,ОТДЕЛОМ УФМС РОССИИ ПО ИРКУТСКОЙ ОБЛАСТИ В ОКТ...,ГОР. ИРКУТСК
1,d099912b-fa46-49ac-8f7b-0f660b184ce6.jpeg,18.08.2010,640-036,РОЩИН,СЕРГЕЙ,МИХАЙЛОВИЧ,11.02.1990,МУЖ.,63 10 509663,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО САРАТОВСКОЙ ОБЛ. В Г...,С. ПОТЬМА РТИЩЕВСКИЙ Р-ОН САРАТОВСКАЯ ОБЛ.
2,7d8c2607-7bd0-47c9-8011-e035a6f34dd4.jpeg,25.06.2007,780-001,ЕГОРОВ,ВАСИЛИЙ,АЛЕКСАНДРОВИЧ,20.04.1987,МУЖ.,40 07 187829,ТП №1 ОТДЕЛА УФМС РОССИИ ПО САНКТ-ПЕТЕРБУРГУ И...,ГОР. ЛЕНИНГРАД
3,de6102e3-2a98-4898-90b2-18f30ba3e912.jpeg,09.08.2017,690-027,КОВАЛЁВ,ЕВГЕНИЙ,ДМИТРИЕВИЧ,13.06.1992,МУЖ.,28 17 540959,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО ТВЕРСКОЙ ОБЛАСТИ В О...,ГОР. ОСТАШКОВ ТВЕРСКАЯ ОБЛ.
4,27f2fdc8b1ffc41d481b57ab8b568917.jpeg,06.07.2022,630-027,ДЖУМАЕВ,АХРОМИДДИН,САЛОМУДИНОВИЧ,30.04.1990,МУЖ.,36 22 110830,ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛАСТИ,С. АЙНИ КУМСАНГИРСКИЙ Р-Н КУРГАН-ТЮБИНСКАЯ ОБЛ...
5,3710bc70ed198d96abb3f7e8e373276c.jpeg,15.02.2002,502-015,ОПРЯ,ДМИТРИЙ,СТЕПАНОВИЧ,07.08.1979,МУЖ.,46 02 527213,ЛОБНЕНСКИМ ОВД МЫТИЩИНСКОГО УВД МОСКОВСКОЙ ОБЛ...,Г. ЛОБНЯ МОСКОВСКОЙ ОБЛ.
6,c10be491ce2c9f2a29747b88ad232b08.jpeg,20.09.2007,770-074,КАЛИНКИН,АЛЕКСАНДР,МИХАЙЛОВИЧ,07.11.1984,МУЖ.,45 09 194829,ОТДЕЛЕНИЕМ ПО РАЙОНУ ФИЛЕВСКИЙ ПАРК ОУФМС РОСС...,ГОРОД МОСКВА
7,985317e9a09699992b2edcfd5e27aff5.jpeg,31.05.2022,460-012,ДЗЕКИЛВ,АЛЕКСЕЙ,ВАСИЛЬЕВИЧ,30.04.1977,МУЖ.,38 22 393737,УМВД РОССИИ ПО КУРСКОЙ ОБЛАСТИ,Г. ЖЕЛЕЗНОГОРСК КУРСКАЯ ОБЛ.
8,582b6e51f655e69e33dcbb6049cae588.jpeg,01.04.2002,162-013,БАШАН,АЛЕКСЕЙ,АЛЕКСАНДРОВИЧ,26.07.1981,МУЖ.,92 02 983452,ЭЛЕКТРОТЕХНИЧЕСКИМ ОВД ГОР. НАБЕРЕЖНЫЕ ЧЕЛНЫ Р...,ГОР. КАЗАНЬ
9,c20e6682-54c4-4c26-949b-cb5d5fa8ea39.jpeg,25.12.2012,660-009,МОШКИН,ИГОРЬ,НИКОЛАЕВИЧ,19.04.1967,МУЖ.,65 12 538322,ОУФМС РОССИИ ПО СВЕРДЛОВСКОЙ ОБЛ. В ЧКАЛОВСКОМ...,ГОР. СВЕРДЛОВСК


In [68]:
df3 = df2.reindex(columns=df1.columns)
df3 = df3.sort_values(by='file_name').reset_index(drop=True)
df3['issued_by_whom'] = df3['issued_by_whom'].astype(str)
df3['issued_by_whom'] = df3['issued_by_whom'].str.strip()
df3['place_of_birth'] = df3['place_of_birth'].astype(str)
df3['place_of_birth'] = df3['place_of_birth'].str.strip()
df3['gender'] = df3['gender'].astype(str)
df3['gender'] = df3['gender'].str.strip()
df3

,file_name,unit_code,date_of_issue,surname,first_name,patronymic,gender,date_of_birth,issued_by_whom,place_of_birth,series_and_number
0,0c5cd9f2-96c9-43ed-9008-f6de3b1a8a8c.jpeg,380-003,02.07.2013,НЫРКО,АНАСТАСИЯ,МАТВЕЕВНА,ЖЕН.,07.05.1991,ОТДЕЛОМ УФМС РОССИИ ПО ИРКУТСКОЙ ОБЛАСТИ В ОКТ...,ГОР. ИРКУТСК,25 13 856830
1,177de6d27597c51ac801fe8bab8e0931.jpeg,230-029,13.01.2012,БАТИЩЕВ,ВЯЧЕСЛАВ,ВЯЧЕСЛАВОВИЧ,МУЖ.,19.12.1991,ОУФМС РОССИИ ПО КРАСНОДАРСКОМУ КРАЮ В БЕЛОГЛИН...,ГОР. МАЙКОП РЕСП. АДЫГЕЯ,03 11 968450
2,27f2fdc8b1ffc41d481b57ab8b568917.jpeg,630-027,06.07.2022,ДЖУМАЕВ,АХРОМИДДИН,САЛОМУДИНОВИЧ,МУЖ.,30.04.1990,ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛАСТИ,С. АЙНИ КУМСАНГИРСКИЙ Р-Н КУРГАН-ТЮБИНСКАЯ ОБЛ...,36 22 110830
3,3672a1e729a3e5371b9ec45c5a91c297.jpeg,632-010,28.09.2002,РУБАНОВ,ДМИТРИЙ,ГЕОРГИЕВИЧ,МУЖ.,20.11.1979,ОКТЯБРЬСКИМ РОВД ГОРОДА САМАРЫ,Г. КУЙБЫШЕВ,36 02 551539
4,3710bc70ed198d96abb3f7e8e373276c.jpeg,502-015,15.02.2002,ОПРЯ,ДМИТРИЙ,СТЕПАНОВИЧ,МУЖ.,07.08.1979,ЛОБНЕНСКИМ ОВД МЫТИЩИНСКОГО УВД МОСКОВСКОЙ ОБЛ...,Г. ЛОБНЯ МОСКОВСКОЙ ОБЛ.,46 02 527213
5,44ab4177f18cd5dd8a898c852c5fd2c4.jpeg,550-038,24.07.2009,ГУДЫМ,МАКСИМ,ВАЛЕРЬЕВИЧ,МУЖ.,30.09.1984,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО ОМСКОЙ ОБЛАСТИ В ЧЕР...,С. МИХАЙЛОВКА ЧЕРЛАКСКОГО Р-НА ОМСКОЙ ОБЛ.,52 09 789231
6,5139349d9409170acd30abcf996f09cf.jpeg,640-011,15.06.2015,БАЛТАДЖЯН,АРКАДИЙ,СЕРГЕЕВИЧ,МУЖ.,04.07.1956,ОТДЕЛОМ УФМС РОССИИ ПО САРАТОВСКОЙ ОБЛАСТИ В Г...,ГОР. ТБИЛИСИ Г. ССР,63 15 160426
7,582b6e51f655e69e33dcbb6049cae588.jpeg,162-013,01.04.2002,БАШАН,АЛЕКСЕЙ,АЛЕКСАНДРОВИЧ,МУЖ.,26.07.1981,ЭЛЕКТРОТЕХНИЧЕСКИМ ОВД ГОР. НАБЕРЕЖНЫЕ ЧЕЛНЫ Р...,ГОР. КАЗАНЬ,92 02 983452
8,68418534-4c5f-43db-8277-37b9596d31ae.jpeg,360-004,07.12.2017,КУРАСОВ,АЛЕКСАНДР,ПЕТРОВИЧ,МУЖ.,06.04.1971,ОТДЕЛОМ УФМС РОССИИ ПО ВОРОНЕЖСКОЙ ОБЛАСТИ В К...,С. Б-ПЕСКИ ЭРТИЛЬСКОГО Р-НА ВОРОНЕЖСКОЙ ОБЛ.,20 17 158065
9,7d8c2607-7bd0-47c9-8011-e035a6f34dd4.jpeg,780-001,25.06.2007,ЕГОРОВ,ВАСИЛИЙ,АЛЕКСАНДРОВИЧ,МУЖ.,20.04.1987,ТП №1 ОТДЕЛА УФМС РОССИИ ПО САНКТ-ПЕТЕРБУРГУ И...,ГОР. ЛЕНИНГРАД,40 07 187829


In [69]:

df4 = df1.eq(df3)
df4 = df4.replace({True: 1, False: 0})
df4 = df4.astype(int)
df4['%'] = df4.iloc[:, 1:].sum(axis=1)/10*100
sum_row = pd.DataFrame(df4.iloc[:, :-1].sum()).T/21*100
sum_row.columns = df4.columns[:-1]
df4 = df4.append(sum_row, ignore_index=True)
pd.set_option('display.max_columns', None)
df4

,file_name,unit_code,date_of_issue,surname,first_name,patronymic,gender,date_of_birth,issued_by_whom,place_of_birth,series_and_number,%
0,1.0,1.0,1.00000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.00000,1.000000,100.0
1,1.0,1.0,0.00000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.00000,1.000000,90.0
2,1.0,1.0,1.00000,1.000000,0.000000,1.000000,1.0,1.0,1.000000,1.00000,1.000000,90.0
3,1.0,1.0,1.00000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.00000,1.000000,100.0
4,1.0,1.0,1.00000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.00000,0.000000,90.0
5,1.0,1.0,1.00000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.00000,1.000000,100.0
6,1.0,1.0,1.00000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.00000,1.000000,100.0
7,1.0,1.0,1.00000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.00000,1.000000,100.0
8,1.0,1.0,1.00000,1.000000,1.000000,1.000000,1.0,1.0,1.000000,1.00000,1.000000,100.0
9,1.0,1.0,1.00000,1.000000,1.000000,1.000000,1.0,1.0,0.000000,1.00000,1.000000,90.0


In [70]:
df4[df4['%']==100.0]

,file_name,unit_code,date_of_issue,surname,first_name,patronymic,gender,date_of_birth,issued_by_whom,place_of_birth,series_and_number,%
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
12,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
15,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0
19,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0


In [71]:
len(df4[df4['%']==100.0])

10

In [72]:
df4[df4['%']==80.0]

,file_name,unit_code,date_of_issue,surname,first_name,patronymic,gender,date_of_birth,issued_by_whom,place_of_birth,series_and_number,%


In [73]:
rows, cols = np.where(df4 == 0)
# Вывод значений ячеек
for row, col in zip(rows, cols):
    file_name = df1.iloc[row, 0]                                                # Имя файла берем из первого столбца
    print(f'File name: {file_name}', f'Row: {row}, Column: {col}', f'df1 value: {df1.iloc[row, col]}', f'df3 value: {df3.iloc[row, col]}')
    

File name: 177de6d27597c51ac801fe8bab8e0931.jpeg Row: 1, Column: 2 df1 value: 73.01.2012 df3 value: 13.01.2012
File name: 27f2fdc8b1ffc41d481b57ab8b568917.jpeg Row: 2, Column: 4 df1 value: АХОМИДИН df3 value: АХРОМИДДИН
File name: 3710bc70ed198d96abb3f7e8e373276c.jpeg Row: 4, Column: 10 df1 value: поле распознано не полностью46024527213 df3 value: 46 02 527213
File name: 7d8c2607-7bd0-47c9-8011-e035a6f34dd4.jpeg Row: 9, Column: 8 df1 value: ТП №1 ОТДЕЛА УФМС РОССИИ ПО АДМИРАЛТЕЙСКОМ Р-НЕ ГОР. САНКТ-ПЕТЕРБУРГА df3 value: ТП №1 ОТДЕЛА УФМС РОССИИ ПО САНКТ-ПЕТЕРБУРГУ И ЛЕНИНГРАДСКОЙ ОБЛ. В АДМИРАЛТЕЙСКОМ Р-НЕ ГОР. САНКТ-ПЕТЕРБУРГА
File name: 985317e9a09699992b2edcfd5e27aff5.jpeg Row: 11, Column: 3 df1 value: ДЗЕКИЛЕВ df3 value: ДЗЕКИЛВ
File name: ab249a02ee45abcc2e4ec628a60c660e.jpeg Row: 13, Column: 5 df1 value: nan df3 value: 
File name: c10be491ce2c9f2a29747b88ad232b08.jpeg Row: 14, Column: 9 df1 value: Н ГОРОД МОСКВА df3 value: ГОРОД МОСКВА
File name: c20e6682-54c4-4c26-949b-cb5d5fa8e